# Reproducibility in Tensorflow/PyTorch/JAX

If you are reading this, I am probably dead 💀 ...

![AI](../images/posts/reproducibility_meme.JPG)

FFrom looking for answers on how to make it repeatable and writing them here. So please keep in mind that this post is not finished as there are still unresolved topics about reproducibility that I will try to trace 🕵. If you see any mistakes please email me with an explanation and I will try to fix it. So after this brief introduction shall we begin?

Reproducibility is a crucial aspect of scientific research, as it allows others to verify and build upon previous findings. This is especially important in the field of machine learning, where reproducibility allows for more accurate comparisons between different methods and algorithms. In this blog, we will explore the issue of reproducibility in the popular machine learning frameworks [Jax](https://jax.readthedocs.io/en/latest/), [Tensorflow](https://www.tensorflow.org), and [Pytorch](https://pytorch.org). We will discuss the challenges and solutions for achieving reproducibility in these frameworks, as well as the benefits of using reproducible methods in machine learning research.

It's worth noting that this blog post will not cover reproducibility in distributed learning environments. I will cover that topic in a separate blog post 📜 focused on distributed learning and reproducibility.

In industry, machine learning engineers rely on reproducible results every time they run code. This allows them to improve upon their models and explore different hyperparameter settings. In the research field, reproducibility is especially important for accurately comparing results to those published in papers. I hope you know that if you want to properly compare your results with those of your work, it is necessary to run the same data loaders and datasets as the original researchers, and even better (in my opinion also essential) to run and compare your solution locally on their pipelines. In this blog we will delve into why reproducibility is so important in both industry and research. **Remember that we are talking about results, not performance, as these can be more easily reproduced.**


## Randomness in deep learning

In deep learning, randomness can come from several sources. For example, when initializing the weights of a neural network, random values are often used to break symmetry and allow for better optimization. Randomness can also come from the stochastic gradient descent (SGD) algorithm, which uses random samples from the training dataset to update the weights. Basically initialization weights, processes that are stochastic dropout for example and as well as data loaders are the jey point you need to focus to have deterministic results. At least on paper but you need to remember that you often work on different environments which also should be reproduced package version, python version etc because maybe in some version they change the behavior of some functions. Lastly, device you will **a lot** about how it is difficult to reproduce on different devices GPU / CPU and across different hardwares also 😵‍💫.

Randomness is an inherent aspect of deep learning, and it can come from a number of sources. For instance, when initializing the weights of a neural network, it is common to use random values to break symmetry and facilitate better optimization. Similarly, the stochastic gradient descent (SGD) algorithm relies on random samples from the training dataset to update the weights, introducing another source of randomness.

To achieve reproducibility in deep learning, it is important to consider the sources of randomness in your model and how they can be controlled. Basic process that introduce randomness in pipelines are:
- initialization weights,
- datasets and data loaders,
- stochastic operation (such as dropout) that on paper are not deterministic.
However, it is also important to consider the impact of the environment on reproducibility. This includes the versions of packages and libraries, the version of Python, and the device (e.g. GPU or CPU) being used. Different hardware and software configurations can lead to variations in the behavior of certain functions, which can affect reproducibility 😵‍💫.

### Environment reproducibility

To ensure reproducibility in your deep learning projects, it is important to carefully manage your environment. Here are some tips for making your environment reproducible:
1. Use a tool like [pyenv](https://realpython.com/intro-to-pyenv/) to manage your Python version. This will ensure that you are using the same version of Python across different environments.
2. Create virtual environments for your projects using tools like [venv](https://docs.python.org/3/library/venv.html), [Poetry](https://python-poetry.org), [Conda](https://conda-forge.org), or [pdm](https://github.com/pdm-project/pdm). This will allow you to isolate your project dependencies and avoid conflicts with other packages. To create `venv` just run in shell `python -m venv venv` and voilà. To activate it run `source venv/bin/activate`.
3. Keep track of the packages and their versions that you use in your virtual environments. You can use the `pip freeze` command to generate a `requirements.txt` file that lists all of the packages and their versions `pip freeze > requirements.txt`. This will help you recreate the exact same environment later on running `pip install -r requirements.txt`.
By keeping track of your environment, you can minimize the impact of differences in package versions and other factors that might lead to variations in your results. This will help you achieve more reproducible results in your deep learning projects.
4. Running on the GPU also requires having the same version of CUDA, the cuDNN version, to be able to reproduce the results even on the same GPU.

### Random number generators (PRNG)

Random number generators (RNGs) are an important component of machine learning frameworks like Pytorch, Tensorflow, and Jax, as well as scientific computing libraries like [Numpy](https://numpy.org). These RNGs use pseudorandom number generators (PRNGs) to produce sequences of numbers that are statistically close to random.

PRNGs are based on algorithms that generate a sequence of numbers using a fixed starting point, called the seed 🌱. The generated numbers are determined by the seed and the algorithm used, so they will be the same each time the code is run with the same seed. This can be useful for reproducibility, as it allows you to obtain the same results by setting the same seed. However, it also means that the generated numbers are not truly random and may not be suitable for certain applications. I advise to look into this lecture from [the university of Utah](https://www.math.utah.edu/~alfeld/Random/Random.html) and this [paper](http://www.thesalmons.org/john/random123/papers/random123sc11.pdf).

There are various algorithms used by PRNGs, including Monte Carlo techniques and counter-based RNGs. The Python `random` module uses the [Mersenne Twister algorithm](https://www.sciencedirect.com/topics/computer-science/mersenne-twister), which uses a state of `624` integers to generate a sequence of seemingly random numbers. Numpy's `numpy.random` and Pytorch's `torch.rand` packages also use the Mersenne Twister algorithm. Tensorflow's `tensorflow.random` package uses the [Philox algorithm](https://numpy.org/doc/stable/reference/random/bit_generators/philox.html), which is based on counter-based RNGs. Jax's `jax.random` package uses the [ThreeFry](https://bashtage.github.io/randomgen/bit_generators/threefry.html), which is also based on counter-based RNGs algorithm.

It is important to keep in mind that these RNGs are not truly random, and they may not be suitable for all applications. If you need truly random numbers, you may need to use a hardware-based RNG, such as a physical device that generates random numbers based on quantum phenomena.


### CUDA, cuDNN, cuBLAS,

When working with GPU you heard about CUDA for sure. [CUDA](https://pl.wikipedia.org/wiki/CUDA) is a parallel computing platform and programming model developed by NVIDIA for general-purpose computations on graphics processing units (GPUs). It allows developers to use the power of GPUs to accelerate their applications, and it is widely used in machine learning and other fields.

[cuDNN](https://blog.roboflow.com/what-is-cudnn/) (CUDA Deep Neural Network library) is a GPU-accelerated library for deep learning developed by NVIDIA. It provides highly optimized implementations of common deep learning operations, such as convolutions and matrix multiplications, which can significantly improve the performance of deep learning models on GPUs.

[cuBLAS](https://docs.nvidia.com/cuda/cublas/index.html) (CUDA Basic Linear Algebra Subprograms) is a GPU-accelerated library for linear algebra operations developed by NVIDIA. It provides optimized implementations of common linear algebra operations, such as matrix multiplications and decompositions, which can accelerate the performance of many machine learning algorithms.

One of the challenges of reproducibility in machine learning is the use of these GPU-accelerated libraries, which can lead to differences in the behavior of the code on different GPU hardware.

To mitigate these issues, it is important to carefully specify the GPU hardware and drivers that you are using, and to ensure that the same hardware and drivers are used when running the code. You can also consider using a GPU cloud service, which can provide a consistent hardware environment for running your code. However, it is worth noting that even with these precautions, there may still be some variability in the results due to differences in the underlying hardware.

### XLA

XLA (Accelerated Linear Algebra) is a domain-specific compiler for linear algebra developed by Google that is used in the Tensorflow machine learning framework and JAX. It is designed to optimize the performance of linear algebra operations, such as matrix multiplications and decompositions, by generating efficient code for different hardware architectures, including CPUs, GPUs, and Tensor Processing Units (TPUs).

One of the benefits of using XLA is that it can improve the performance of Tensorflow models by generating highly optimized code for different hardware architectures. This can be particularly useful for large-scale machine learning tasks, where the performance of the model can have a significant impact on the time required to train and evaluate the model.

XLA is also designed to be deterministic, which can help with reproducibility in machine learning. By generating the same code for a given set of inputs, XLA can help ensure that the results of a Tensorflow and Jax model are consistent across different runs and hardware platforms. However, it is worth noting that there may still be some variability in the results due to differences in the underlying hardware and other factors.

## Well, after this long introduction, perhaps it's time to code 👩‍💻

Firstly we will set some variables. We will only set `CUDA_VISIBLE_DEVICES` to ensure that we are working on one device and `CUBLAS_WORKSPACE_CONFIG`. `CUBLAS_WORKSPACE_CONFIG` maybe needed by Pytorch becouse, bit-wise reproducibility is not guaranteed across toolkit versions because the implementation might differ due to some implementation changes. You will get to choose `:16:8` (may limit overall performance) or `:4096:8` (will increase library footprint in GPU memory by approximately 24MiB). Any of those settings will allow for deterministic behavior even with multiple concurrent streams sharing a single cuBLAS handle.

To ensure reproducibility in our deep learning code, we need to set some variables that will control the behavior of our model. Specifically, we will set the `CUDA_VISIBLE_DEVICES` variable to ensure that we are using a specific GPU device, and the `CUBLAS_WORKSPACE_CONFIG` variable to configure cuBLAS, as bit-wise reproducibility is not guaranteed across toolkit versions.

The `CUDA_VISIBLE_DEVICES` variable is used to specify which GPU devices are visible to the code. By setting this variable, we can ensure that our code is only using a single GPU device. The `CUBLAS_WORKSPACE_CONFIG` variable can take two values: `:16:8` and `:4096:8`. The `:16:8` setting may limit overall performance, but it allows for deterministic behavior even with multiple concurrent streams sharing a single cuBLAS handle. The `:4096:8` setting will increase the library footprint in GPU memory by approximately 24MiB, but it also allows for deterministic behavior.

In [ ]:
# set gpu id to test it on one device
%env CUDA_VISIBLE_DEVICES=0
# You may need to set this variables by PyTorch. It may limit overall performance so be aware of it.
%env CUBLAS_WORKSPACE_CONFIG=:16:8

After that, we can import all necessary libraries and set up the environment. But first we will import not machine learning libraries that are also necessary.

In [ ]:
import os
import time
import pickle

Although we already set some env variables we need to set some additional variables.

In addition to the randomness provided by the random module, Python uses a random seed for its hashing algorithm, which affects how objects are stored in sets and dictionaries. This must happen the same way every time in order for GerryChain runs to be repeatable. The way to accomplish this is to set the [environment variable](https://docs.python.org/3.3/using/cmdline.html) `PYTHONHASHSEED`.

By utilizing cuDNN, Tensorflow/JAX/PyTorch frameworks can take advantage of the specialized hardware and algorithms provided by NVIDIA GPUs to speed up the training and inference of deep learning models. However the downside of this is lack of reproducibility in some cases so we need to not disable it as it is integrated with CUDA. In Tensorflow now you have a function to make all determinisitc as possible but in previous version you need to do something: In version 1.14, 1.15 and 2.0 you do it by setting `TF_CUDNN_DETERMINISTIC` to `1`, in 2.1-2.8 version you set `TF_CUDNN_USE_FRONTEND` and `TF_DETERMINISTIC_OPS` to `1`. As I am working now on newer version of Tensorflow I am not setting those variables. You can read more about it [here](https://github.com/NVIDIA/framework-determinism/blob/master/doc/tensorflow_status.md).

However, for Jax setting `TF_CUDNN_DETERMINISTIC` to `1` allowed me to get reproduciable results so this variable for Jax should be used

In [ ]:
# All frameworks
os.environ['PYTHONHASHSEED'] = '0'
# Jax
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

And finally time for importing our main machine learning modules

In [ ]:
# main imports
import random
import jax
import torch
import numpy as np
import tensorflow as tf

# additional imports
import torchvision
import jax.numpy as jnp
from flax import linen as nn
from flax import serialization as flax_serialization
from flax.training import train_state, checkpoints
import optax

In order to ensure that our pipeline is reproducible, the final step is to adjust the parameters in our framework.

One important consideration when setting these parameters is the use of thread parallelism. Thread parallelism refers to the ability of a computer program to execute multiple threads concurrently. In the context of deep learning, this technique can be used to speed up the training process by allowing different threads to work on different parts of the training data at the same time. This can help to reduce the overall training time and improve the efficiency of the learning algorithm. However this also may influence how reproducible are our results.

### Tensorflow

To ensure reproducible results in Tensorflow, it's important to properly configure the threading behavior of the runtime. The `tf.config.threading` module provides APIs for configuring the threading behavior of the TensorFlow runtime. The `tf.config.threading.set_inter_op_parallelism_threads` function specifies the number of threads to use for parallel execution of independent operations (also known as "inter-op" parallelism), while `tf.config.threading.set_intra_op_parallelism_threads` specifies the number of threads to use for parallel execution of operations within a single op (also known as "intra-op" parallelism).

In addition to configuring threading behavior, Tensorflow 2.8 introduced a single function for enabling deterministic operations: `tf.config.experimental.enable_op_determinism()`. According to the Tensorflow documentation, when this function is called, "TensorFlow ops will be deterministic."

For more information on reproducibility in Tensorflow, you can also check out the resources linked in the original [page]((https://www.tensorflow.org/api_docs/python/tf/config/experimental/enable_op_determinism)): the NVIDIA framework determinism [documentation](https://github.com/NVIDIA/framework-determinism/blob/master/doc/tensorflow.md) and the tensorflow-determinism package on [PyPI](https://pypi.org/project/tensorflow-determinism/).

### Pytorch
In the case of thread parallelism, you can avoid setting everything to just one thread, as Pytorch gives a nice solution to this problem, which you will explore later.

There are also a few configs related to cuDNN that you may want to set in order to ensure reproducibility:
- `torch.backends.cudnn.deterministic`: A `bool` that, if set to True, causes cuDNN to only use deterministic convolution algorithms.
- `torch.backends.cudnn.benchmark`: A `bool` that, if set to True, causes cuDNN to benchmark multiple convolution algorithms and select the fastest.

In addition to these configs, Pytorch provides a function called `torch.use_deterministic_algorithms`, which "configures PyTorch to use deterministic algorithms instead of nondeterministic ones where available, and to throw an error if an operation is known to be nondeterministic (and without a deterministic alternative)." This can help to ensure reproducibility of your results.

For more information on reproducibility in Pytorch, you can refer to the Pytorch [documentation](https://pytorch.org/docs/stable/notes/randomness.html) on randomness and the NVIDIA framework determinism [documentation](https://github.com/NVIDIA/framework-determinism/blob/master/doc/pytorch.md), which includes a section specifically on Pytorch.

### JAX

Jax is designed to be deterministic, as long as everything is implemented correctly (To my knowledge). In most cases, XLA used by Jax is deterministic on its own. However, there is an issue with cuDNN that can impact the reproducibility of Jax programs. To address this issue, you can set the `TF_CUDNN_DETERMINISTIC` environment variable to `1`, which will make cuDNN deterministic for Jax.

It's worth noting that `TF_CUDNN_DETERMINISTIC` is a Tensorflow environment variable, even though Jax is a separate library. This is because Jax is developed by Google, the same company that developed Tensorflow. As a result, there is some overlap between the two libraries, and you may need to use Tensorflow-specific tools to ensure reproducibility in your Jax programs my padawans 🤓.

In [ ]:
################ Tensorflow ################
# set the number of threads running on the CPU
tf.config.threading.set_intra_op_parallelism_threads(1)
tf.config.threading.set_inter_op_parallelism_threads(1)

# set rest of operation to be deterministic
tf.config.experimental.enable_op_determinism()

################ Pytorch ###################
# cudnn settings
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# set rest of operation to be deterministic
torch.use_deterministic_algorithms(mode=True, warn_only=True)

Check which device we shall be using.

In [ ]:
print("Devices we will use")
try:
    print(torch.cuda.get_device_name(), torch.cuda.device_count())
    print(tf.config.list_physical_devices("GPU"))
    print(jax.devices())
except:
    print("Well no GPU for you my friend")

### What to keep in mind when creating a pipeline

#### Set seed 🌱

Now let's create an array of 2 by 2 random numbers from a Gaussian normal distribution.

In [ ]:
# python numbers
python_array = [[random.random() for _ in range(2)] for i in range(2)]
print(f"Python output: {python_array}")
# numpy
numpy_array = np.random.rand(2, 2)
print(f"Numpy output: {numpy_array}")
# pytorch
pytorch_array = torch.rand(2, 2)
print(f"PyTorch output: {pytorch_array}")
# tensorflow
tf_array = tf.random.uniform((2, 2))
print(f"Tensorflow output: {tf_array}")

Try to run this code three times and you will se that each time you will get different results.

So now let's set seed. **Remember** you need to set seed for all the used packages and better to always set seed for `random` and `numpy` packages as you don't know what package generate random numbers in some frameworks. Also you will now see that we will also test JAX as it is the only package where you need to always provide RNGs to every random number creation so you always have reproducible results (if you will not screw up somehow), well this solution have some annoying minuses.

To demonstrate the importance of setting seeds for reproducibility, let's consider a simple example where we generate random numbers using different packages. If you run this code multiple times, you'll notice that the results are different each time. This is because the default behavior of these packages is to generate random numbers based on the current system time, which means that the numbers will be different every time the code is run (I don't know if it is True chatGPT told me that, but results are the same).

To ensure reproducibility, it's important to set seeds for all the packages that generate random numbers. This is especially important for the `random` and `numpy` packages, as you may not always know which package is being used to generate random numbers in a particular framework.

One exception to this rule is JAX, which requires you to provide a random number generator (RNG) for every random number generation. This means that you must always keep track of this variable holding the RNG. While this solution is effective, it can be a little cumbersome as you have to provide an RNG for every random number generation.


In [ ]:
# Seed for the random number generator
seed = 0
# seting PRNG for random package
random.seed(seed)
# seting PRNG for numpy
np.random.seed(seed)
# seting PRNG for pytorch
torch.manual_seed(seed)
# if you are on cuda you need to set the seed for the cuda PRNG
torch.cuda.manual_seed(seed)
# seting PRNG for tensorflow
tf.random.set_seed(seed)
# In tensorflow you can set seed for Python, NumPy and TensorFlow with one function
# tf.keras.utils.set_random_seed(seed)
# seting PRNG for jax
rng = jax.random.PRNGKey(seed)

And now it's time to test it.

In [ ]:
# python numbers
python_array = [[random.random() for _ in range(2)] for i in range(2)]
print(f"Python output: {python_array}")
# numpy
numpy_array = np.random.rand(2, 2)
print(f"Numpy output: {numpy_array}")
# pytorch
pytorch_array = torch.rand(2, 2)
print(f"PyTorch output: {pytorch_array}")
# tensorflow
tf_array = tf.random.uniform((2, 2))
print(f"Tensorflow output: {tf_array}")
# JAX
jax_array = jax.random.uniform(rng, (2, 2))
print(f"JAX output: {jax_array}")

If you run the code with those seeds multiple times you'll notice that the results are <span style="color:green">consistent</span> each time. This demonstrates the effectiveness of setting seeds to ensure reproducibility.

It's worth noting that if you re-run the code in a notebook environment, the results may not be consistent because the code is being run in separate cells. To properly test the reproducibility of the code, you should run it multiple times as a single script or restart the notebook each time.

In the case of JAX, it's important to keep track of your RNGs and use them correctly in order to ensure reproducibility. For more information on how to use RNGs in JAX and the advantages of this approach, you can refer to the JAX [documentation](https://jax.readthedocs.io/en/latest/jax-101/05-random-numbers.html) on random numbers.

### Datasets and Dataloader

As we mentioned earlier, the optimizer updates our deep learning model based on incoming data. If we can make this incoming data deterministic, we can achieve reproducible results from our optimizer.

To ensure reproducibility for datasets and data loaders in Pytorch and Tensorflow, you can use separate generators for each. This will ensure that the same dataset and data loader flow are generated each time the code is run, which can be useful for testing and debugging, as well as for ensuring that the results of a machine learning experiment are reproducible.

It's important to note that JAX does not have a built-in data loader. However, you can still use the techniques described here to ensure reproducibility for your datasets in JAX.

Next, let's take a closer look at the typical usage of datasets and data loaders in Pytorch and Tensorflow.

In [ ]:
# Create dummy dataset with pytorch
class DummyPytorchDataset(torch.utils.data.Dataset):
    def __init__(self, shape=(4, 2)):
        self.data = torch.rand(*shape)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]


# We create two datasets, one for training and one for testing
pytorch_dataset_train = DummyPytorchDataset()
pytorch_dataset_test = DummyPytorchDataset()

# Create dataloaders
# Train dataloader should be shuffled and test dataloader should not be shuffled
pytorch_dataloader_train = torch.utils.data.DataLoader(
    pytorch_dataset_train, batch_size=2, shuffle=True 
)
pytorch_dataloader_test = torch.utils.data.DataLoader(
    pytorch_dataset_test, batch_size=2, shuffle=False
)

In [ ]:
# Create dummy dataset with tensorflow
tf_dataset_train = (
    tf.data.Dataset.from_tensor_slices(tf.random.normal((4, 2)))
    .cache()
    .shuffle(4)
    .batch(2)
    .prefetch(tf.data.AUTOTUNE)
)
tf_dataset_test = (
    tf.data.Dataset.from_tensor_slices(tf.random.normal((4, 2)))
    .batch(2)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

# Tensorflow does not have a dataloader, but we can iterate over the dataset or create an iterator
# In this case we will use build in function to iterate over the dataset as numpy arrays

Now have a look at the data for the two epochs.

In [ ]:
for epoch in range(2):
    print(f"Epoch {epoch}")
    print("Train dataset with shuffle")
    for i, batch in enumerate(pytorch_dataloader_train):
        print(f"Batch id {i} with data {batch}")

    print("Test dataset without shuffle")
    for i, batch in enumerate(pytorch_dataloader_test):
        print(f"Batch id {i} with data {batch}")

In [ ]:
for epoch in range(2):
    print(f"Epoch {epoch}")
    print("Train dataset with shuffle")
    for i, batch in enumerate(tf_dataset_train.as_numpy_iterator()):
        print(f"Batch id {i} with data {batch}")

    print("Test dataset without shuffle")
    for i, batch in enumerate(tf_dataset_test.as_numpy_iterator()):
        print(f"Batch id {i} with data {batch}")

If you run the code with previous settings and then restart it, you'll get the same results. However, what if you want to go to a specific checkpoint in your dataset and start from there? If you just load the model weights and optimizers and run the code, you'll get the same data as from the beginning of the first epoch, not the saved data from the checkpoint.

To save the state of your data loaders and reproduce the data from a specific point, you can use different approaches depending on the framework you're using. In Tensorflow, I don't know if there is currently way to save the state of a data loader, so I opened this [issue](https://github.com/tensorflow/tensorflow/issues/58925) to find out.

In Pytorch, you can seed your workers and pass a `torch.Generator` to the `Dataloader`. This generator will handle shuffling and data pre-processing randomness, and it can be used to save and load the state of the generator. This will allow you to reproduce your data from any point in the training process. You can find more information on this technique in the Pytorch [documentation](https://pytorch.org/docs/stable/data.html#data-loading-randomness).

Here's an example of how you can use a torch.Generator to save and load the state of a Pytorch data loader:

In [ ]:
# Take care that each worker has consistent seed
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

# Now we create a generator and pass it to the dataloader
g = torch.Generator()
g.manual_seed(seed)

pytorch_dataloader_train = torch.utils.data.DataLoader(
    pytorch_dataset_train, batch_size=2, shuffle=True, worker_init_fn=seed_worker, generator=g,
)

And now let's see how we can reproduce the same batches of data when shuffling.

In [ ]:
pytorch_rng = g.get_state()
for idx, batch in enumerate(pytorch_dataloader_train):
    if idx == 0:
        print(batch)

# now without loading rng state
for idx, batch in enumerate(pytorch_dataloader_train):
    if idx == 0:
        print(batch)

# now we load the rng state
g.set_state(pytorch_rng)
for idx, batch in enumerate(pytorch_dataloader_train):
    if idx == 0:
        print(batch)

As we mentioned earlier, the state of the random number generator (RNG) is important for reproducing the behavior of a dataset at a specific epoch. Without saving the RNG of the generator, we can't go back to a previous epoch and reproduce the dataset behavior because the shuffling is generated from the RNG, and we need to know its state before shuffling to reproduce it correctly.

To illustrate how to set up reproducible training using different frameworks, let's consider the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset as an example. In the following sections, we'll show you examples of reproducible training in Pytorch, Tensorflow, and JAX, and provide a brief summary of each example.

## PyTorch

In [ ]:
# Ensure once again seed
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# Gnerator for Dataloaders
g_train = torch.Generator()
g_train.manual_seed(seed)

g_test = torch.Generator()
g_test.manual_seed(seed)


# Parameters for training
n_epochs = 4
num_workers = 4
batch_size_train = 32
batch_size_test = 64
learning_rate = 0.01
momentum = 0.5
log_interval = 1000
cpu = False # NOTE Change if you want to do on cpu/cuda
if cpu:
    device = torch.device("cpu") 
else:
    device = torch.device("cuda") 

In [ ]:
# Create dataloaders
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)


train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST(
        "/mnt/data/alzaig/tmp/",
        train=True,
        download=True,
        transform=torchvision.transforms.Compose(
            [
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,)),
            ]
        ),
    ),
    batch_size=batch_size_train,
    shuffle=True,
    num_workers=num_workers,
    worker_init_fn=seed_worker, 
    generator=g_train,
)

test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST(
        "/mnt/data/alzaig/tmp/",
        train=False,
        download=True,
        transform=torchvision.transforms.Compose(
            [
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,)),
            ]
        ),
    ),
    batch_size=batch_size_test,
    shuffle=False,
    num_workers=num_workers,
    worker_init_fn=seed_worker, 
    generator=g_test,
)

In [ ]:
class Net(torch.nn.Module):
    """Net class for mnist example"""

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = torch.nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = torch.nn.Dropout2d()
        self.fc1 = torch.nn.Linear(320, 50)
        self.fc2 = torch.nn.Linear(50, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = torch.nn.functional.relu(torch.nn.functional.max_pool2d(self.conv1(x), 2))
        x = torch.nn.functional.relu(
            torch.nn.functional.max_pool2d(self.conv2_drop(self.conv2(x)), 2)
        )
        x = x.view(-1, 320)
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.dropout(x, training=self.training)
        x = self.fc2(x)
        return torch.nn.functional.log_softmax(x, dim=1)

In [ ]:
# We create a network and an optimizer
network = Net().to(device)
optimizer = torch.optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)

In [ ]:
def train(epoch: int):
    """Train the model

    Args:
        epoch (int): current epoch
    """
    network.train()
    start = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data.to(device))
        loss = torch.nn.functional.nll_loss(output, target.to(device))
        loss.backward()
        optimizer.step()
    
    end_time = time.time() - start            
    print(
        "Train Epoch: {} {:.4f}s\tLoss: {:.6f}".format(
            epoch,
            end_time,
            loss.item(),
            )
        )


def test(epoch: int):
    """Test the model

    Args:
        epoch (int): current epoch
    """
    network.eval()
    test_loss = 0
    correct = 0
    start = time.time()
    with torch.no_grad():
        for data, target in test_loader:
            target = target.to(device)
            output = network(data.to(device))
            test_loss += torch.nn.functional.nll_loss(
                output, target, reduction='sum'
            ).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum().detach().cpu()
    end_time = time.time() - start
    test_loss /= len(test_loader.dataset)
    print(
        "Test set: {:.4f}s Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)".format(
            end_time,
            test_loss,
            correct,
            len(test_loader.dataset),
            100.0 * correct / len(test_loader.dataset),
        )
    )

    # Save the model
    torch.save(
        {
            "epoch": epoch,
            "model_state_dict": network.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "torch_rng": torch.get_rng_state(),
            'torch_cuda_rng': 0 if cpu else torch.cuda.get_rng_state(),
            "numpy_rng": np.random.get_state(),
            "python_state": random.getstate(),
            "generator_dataloader_train": g_train.get_state(),
            "generator_dataloader_test": g_test.get_state()
        },
        f"/mnt/data/alzaig/tmp/pytorch_model_{epoch}.pth",
    )

In [ ]:
# We train the model
test(0)
for epoch in range(1, n_epochs + 1):
    train(epoch)
    test(epoch)

In [ ]:
# Make sure our model is loaded not from cache
del network, optimizer

# create a new network and optimizer
network = Net().to(device)
optimizer = torch.optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)

# Load last checkpoint
checkpoint = torch.load(f"/mnt/data/alzaig/tmp/pytorch_model_{n_epochs}.pth")
epoch_last = checkpoint["epoch"]
network.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
torch.set_rng_state(checkpoint["torch_rng"])
if not cpu:
    torch.cuda.set_rng_state(checkpoint['torch_cuda_rng'])
g_train.set_state(checkpoint["generator_dataloader_train"])
g_test.set_state(checkpoint["generator_dataloader_test"])
np.random.set_state(checkpoint["numpy_rng"])
random.setstate(checkpoint["python_state"])

In [ ]:
test(epoch_last)
for epoch in range(epoch_last + 1, n_epochs + 3):
    train(epoch)
    test(epoch)

In [ ]:
# Make sure our model is loaded not from cache
del network, optimizer

# create a new network and optimizer
network = Net().to(device)
optimizer = torch.optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)

# Load second checkpoint
checkpoint = torch.load(f"/mnt/data/alzaig/tmp/pytorch_model_2.pth")
epoch_last = checkpoint["epoch"]
network.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
torch.set_rng_state(checkpoint["torch_rng"])
if not cpu:
    torch.cuda.set_rng_state(checkpoint['torch_cuda_rng'])
g_train.set_state(checkpoint["generator_dataloader_train"])
g_test.set_state(checkpoint["generator_dataloader_test"])
np.random.set_state(checkpoint["numpy_rng"])
random.setstate(checkpoint["python_state"])

In [ ]:
test(epoch_last)
for epoch in range(epoch_last + 1, n_epochs + 1):
    train(epoch)
    test(epoch)

Summary time:
- Can we reproduce the results by restarting the script for CPU? Answer ✅.
- Can we reproduce the results by restarting the script for the GPU? Answer ✅
- Can we reproduce the results from the checkpoints? Answer ✅
- Can we reproduce results on CPU and GPU (same device)? Answer ❌
- Can we reproduce results on different CPUs? Answer ❌
- Can we reproduce results on different but same GPU? Answer ✅
- Can we reproduce the results on different graphics cards? Answer ❌ (but I will check this on a few different cards)

### Tensorflow 

In [ ]:
# Ensure once again seed
tf.keras.utils.set_random_seed(seed)

# Parameters for training
n_epochs = 4
batch_size_train = 16
batch_size_test = 32
learning_rate = 0.01
momentum = 0.5

In [ ]:
# Create a Tensorflow dataset / Keras dataset
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data(path="/mnt/data/alzaig/tmp/mnist.npz",)
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
# Create Keras model
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Reshape((28,28,-1)),
        tf.keras.layers.Conv2D(filters=10, kernel_size=5),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Conv2D(filters=20, kernel_size=5),
        # sorry I did have small error when I used dropout here so I just removed it
        # It will not change enything just for comparison between architectures it will
        tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(50, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10),
    ]
)

In [ ]:
# Define loss and optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum)

In [ ]:

# Compile the model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=["accuracy"])
# Create checkpoint callback
cp = tf.train.Checkpoint(model=model, optimizer=optimizer)

In [ ]:
model.evaluate(x_test, y_test, verbose=2)
for epoch in range(1, n_epochs + 1):
    print("Epoch: ", epoch)
    model.fit(
        x_train,
        y_train,
        epochs=1,
        workers=1, #you may need to switch to 0 if you have problems with multiprocessing
        batch_size=batch_size_train,
    )
    cp.write(f"/mnt/data/alzaig/tmp/tf_models_{epoch}.h5")
    model.evaluate(x_test, y_test, verbose=2)
    

In [ ]:
# Load last epoch
cp.restore(f"/mnt/data/alzaig/tmp/tf_models_{n_epochs}.h5")

model.evaluate(x_test, y_test, verbose=2)
for epoch in range(n_epochs + 1, n_epochs + 3):
    print("Epoch: ", epoch)
    model.fit(
        x_train,
        y_train,
        epochs=1,
        workers=1, #you may need to switch to 0 if you have problems with multiprocessing
        batch_size=batch_size_train,
    )
    cp.write(f"/mnt/data/alzaig/tmp/tf_models_{epoch}.h5")
    model.evaluate(x_test, y_test, verbose=2)

In [ ]:
# Load second epoch
cp.restore(f"/mnt/data/alzaig/tmp/tf_models_2.h5")

model.evaluate(x_test, y_test, verbose=2)
for epoch in range(2 + 1, n_epochs + 1):
    print("Epoch: ", epoch)
    model.fit(
        x_train,
        y_train,
        epochs=1,
        workers=1, #you may need to switch to 0 if you have problems with multiprocessing
        batch_size=batch_size_train,
    )
    cp.write(f"/mnt/data/alzaig/tmp/tf_models_{epoch}.h5")
    model.evaluate(x_test, y_test, verbose=2)  

Summary time:
- Can we reproduce the results by restarting the script for CPU? Answer ✅.
- Can we reproduce the results by restarting the script for the GPU? Answer ✅
- Can we reproduce the results from the checkpoints? Answer ❌ Due to data loaders not reproducible
- Can we reproduce results on CPU and GPU (same device)? Answer ❌
- Can we reproduce results on different CPUs? Answer ❌
- Can we reproduce results on different but same GPU? Answer ✅
- Can we reproduce the results on different graphics cards? Answer ❌ (but I will check this on a few different cards)

### JAX

For our example of reproducible training in JAX, we'll be using the [Flax](https://flax.readthedocs.io/en/latest/) library to build our deep neural network (DNN). Flax is an overlay library built on top of JAX that provides a more intuitive interface for building DNNs. It does not add any additional functionality to JAX, but it makes it easier to build DNNs using JAX, which was not designed specifically for this purpose.

In [ ]:
# Ensure once again seed
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
rng = jax.random.PRNGKey(seed)

# Gnerator for Dataloaders
g_train = torch.Generator()
g_train.manual_seed(seed)

g_test = torch.Generator()
g_test.manual_seed(seed)

# Parameters for training
n_epochs = 4
num_workers = 4
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5

In [ ]:
class CNN(nn.Module):
  """A simple CNN model."""

  @nn.compact
  def __call__(self, x, training=False):
    x = nn.Conv(features=10, kernel_size=(5, 5))(x)
    x = nn.max_pool(x, window_shape=(2, 2), strides=(2, 2))
    x = nn.relu(x)
    x = nn.Conv(features=20, kernel_size=(5, 5))(x)
    x = nn.Dropout(0.2)(x, deterministic=not training)
    x = nn.max_pool(x, window_shape=(2, 2), strides=(2, 2))
    x = nn.relu(x)
    x = x.reshape((x.shape[0], -1))  # flatten
    x = nn.Dense(features=50)(x)
    x = nn.relu(x)
    x = nn.Dropout(0.2)(x, deterministic=not training)
    x = nn.Dense(features=10)(x)
    return x

In [ ]:
def cross_entropy_loss(logits, labels):
    """Compute the cross-entropy loss given logits and labels.

    Args:
        logits: output of the model
        labels: labels of the data
    """
    labels_onehot = jax.nn.one_hot(labels, num_classes=10)
    return optax.softmax_cross_entropy(logits=logits, labels=labels_onehot).mean()


def compute_metrics_jax(logits, labels):
    """Compute metrics for the model.

    Args:
        logits: output of the model
        labels: labels of the data
    """
    loss = cross_entropy_loss(logits=logits, labels=labels)
    accuracy = jnp.mean(jnp.argmax(logits, -1) == labels)
    return {'loss': loss, 'accuracy': accuracy}


def create_train_state(rng, learning_rate, momentum):
    """Creates initial `TrainState` with `sgd`."""
    cnn = CNN()
    rng, rng_dropout = jax.random.split(rng)
    params = cnn.init(rng, jnp.ones([1, 28, 28, 1]))['params'] # initialize parameters by passing a template image
    tx = optax.sgd(learning_rate, momentum)
    return train_state.TrainState.create(
        apply_fn=cnn.apply, params=params, tx=tx)

In [ ]:
@jax.jit
def train_step(state, image, label, dropout_rng):
  """Train for a single step. Also jit-compiled for speed.
  
  Args:
    params: parameters of the model
    image: input image
    label: label of the image
    dropout_rng: rng for dropout
  """
  def loss_fn(params):
    logits = CNN().apply({'params': params}, x=image, training=True, rngs={'dropout': dropout_rng})
    loss = cross_entropy_loss(logits=logits, labels=label)
    return loss, logits
  grad_fn = jax.grad(loss_fn, has_aux=True)
  grads, logits = grad_fn(state.params)
  state = state.apply_gradients(grads=grads)
  metrics = compute_metrics_jax(logits=logits, labels=label)
  return state, metrics


@jax.jit
def eval_step(params, image, label):
  """Evaluate for a single step. Also jit-compiled for speed.
  
  Args:
    params: parameters of the model
    image: input image
    label: label of the image
  """
  logits = CNN().apply({'params': params}, image)
  return compute_metrics_jax(logits=logits, labels=label)

In [ ]:
def train_epoch(state, train_ds, epoch, rng):
  """Train for a single epoch.
  
  Args:
    params: model parameters
    test_ds: test dataloader
    epoch: current epoch
  """
  batch_metrics = []
  start = time.time()
  for batch_idx, (data, target) in enumerate(train_ds):
    rng, rng_dropout = jax.random.split(rng)
    state, metrics = train_step(state, data, target, rng_dropout)
    batch_metrics.append(metrics)

  # compute mean of metrics across each batch in epoch.
  batch_metrics_np = jax.device_get(batch_metrics)
  epoch_metrics_np = {
      k: np.mean([metrics[k] for metrics in batch_metrics_np])
      for k in batch_metrics_np[0]} # jnp.mean does not work on lists

  end_time = time.time() - start
  print('train epoch: %d, time %.4f,, loss: %.4f, accuracy: %.2f' % (
      epoch, end_time, epoch_metrics_np['loss'], epoch_metrics_np['accuracy'] * 100))
  return state, rng


def eval_model(params, test_ds):
  """Evaluate the model.
  
  Args:
    params: model parameters
    test_ds: test dataloader
  """
  batch_metrics = []
  start = time.time()
  for batch_idx, (data, target) in enumerate(test_ds):
    metrics = eval_step(params, data, target)
    batch_metrics.append(metrics)
    
  # compute mean of metrics across each batch in epoch.
  batch_metrics_np = jax.device_get(batch_metrics)
  metrics = {
      k: np.mean([metrics[k] for metrics in batch_metrics_np])
      for k in batch_metrics_np[0]} # jnp.mean does not work on lists
  end_time = time.time() - start
  summary = jax.tree_util.tree_map(lambda x: x.item(), metrics) # map the function over all leaves in metrics
  return summary['loss'], summary['accuracy'], end_time

In [ ]:
# Transformations applied on each image => bring them into a numpy array
def image_to_numpy(img):
    img = np.array(img, dtype=np.float32)
    img = np.transpose(img, (1, 2, 0))
    return img

# We need to stack the batch elements
def numpy_collate(batch):
    if isinstance(batch[0], np.ndarray):
        return np.stack(batch)
    elif isinstance(batch[0], (tuple,list)):
        transposed = zip(*batch)
        return [numpy_collate(samples) for samples in transposed]
    else:
        return np.array(batch)


def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)


train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST(
        "/mnt/data/alzaig/tmp/",
        train=True,
        download=True,
        transform=torchvision.transforms.Compose(
            [
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,)),
                image_to_numpy
            ]
        ),
    ),
    batch_size=batch_size_train,
    shuffle=True,
    collate_fn=numpy_collate,
    num_workers=num_workers,
    worker_init_fn=seed_worker, 
    generator=g_train,
)

test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST(
        "/mnt/data/alzaig/tmp/",
        train=False,
        download=True,
        transform=torchvision.transforms.Compose(
            [
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,)),
                image_to_numpy
            ]
        ),
    ),
    batch_size=batch_size_test,
    shuffle=False,
    collate_fn=numpy_collate,
    num_workers=num_workers,
    worker_init_fn=seed_worker, 
    generator=g_test,
)

In [ ]:
rng, init_rng = jax.random.split(rng)
state = create_train_state(init_rng, learning_rate, momentum)

In [ ]:
test_loss, test_accuracy, end_time = eval_model(state.params, test_loader)
print(' test epoch: %d, time: %.4f, loss: %.2f, accuracy: %.2f' % (
      0, end_time, test_loss, test_accuracy * 100))

for epoch in range(1, n_epochs + 1):
  state, rng = train_epoch(state, train_loader, epoch, rng)
  # Evaluate on the test set after each training epoch
  test_loss, test_accuracy, end_time = eval_model(state.params, test_loader)
  print(' test epoch: %d, time: %.4f, loss: %.2f, accuracy: %.2f' % (
      epoch, end_time, test_loss, test_accuracy * 100))
  # Save the model parameters
  numpy_rng = np.random.get_state()
  python_state = random.getstate()
  torch_state = torch.get_rng_state() # well only this is required but for the sake of completeness I am adding all
  checkpoints.save_checkpoint(
      ckpt_dir="/mnt/data/alzaig/tmp/jax_checkpoints_{}".format(epoch),
      target={"state": state,
              "epoch": epoch,
              "rng": rng,
              "pytorch_rng": torch_state.numpy(),
              "numpy_rng": numpy_rng,
              "python_state": python_state,
              "generator_dataloader_train": g_train.get_state().numpy(),
              "generator_dataloader_test": g_test.get_state().numpy(),
            },
            step=epoch,
            overwrite=True,
    )

In [ ]:
# Load the model parameters
load_dict = checkpoints.restore_checkpoint(ckpt_dir=f"/mnt/data/alzaig/tmp/jax_checkpoints_{n_epochs}", target=None)
rng = load_dict["rng"]
torch.set_rng_state(torch.tensor(load_dict["pytorch_rng"]))
python_state = list(load_dict["python_state"].values())
python_state[1] = tuple(python_state[1].values())
g_train.set_state(torch.tensor(load_dict["generator_dataloader_train"]))
g_test.set_state(torch.tensor(load_dict["generator_dataloader_test"]))
np.random.set_state(tuple(load_dict["numpy_rng"].values()))
random.setstate(python_state)
epoch_last = load_dict["epoch"]
state = flax_serialization.from_state_dict(state, load_dict['state'])

In [ ]:
test_loss, test_accuracy, end_time = eval_model(state.params, test_loader)
print(' test epoch: %d, time: %.4f, loss: %.2f, accuracy: %.2f' % (
      epoch_last, end_time, test_loss, test_accuracy * 100))

for epoch in range(epoch_last+1, n_epochs + 3):
  state, rng = train_epoch(state, train_loader, epoch, rng)
  # Evaluate on the test set after each training epoch
  test_loss, test_accuracy, end_time = eval_model(state.params, test_loader)
  print(' test epoch: %d, time: %.4f, loss: %.2f, accuracy: %.2f' % (
      epoch, end_time, test_loss, test_accuracy * 100))
  # Save the model parameters
  numpy_rng = np.random.get_state()
  python_state = random.getstate()
  torch_state = torch.get_rng_state() # well only this is required but for the sake of completeness I am adding all
  checkpoints.save_checkpoint(
      ckpt_dir="/mnt/data/alzaig/tmp/jax_checkpoints_{}".format(epoch),
      target={"state": state,
              "epoch": epoch,
              "rng": rng,
              "pytorch_rng": torch_state.numpy(),
              "numpy_rng": numpy_rng,
              "python_state": python_state,
              "generator_dataloader_train": g_train.get_state().numpy(),
              "generator_dataloader_test": g_test.get_state().numpy(),
            },
            step=epoch,
            overwrite=True,
    )

In [ ]:
# Load the second model parameters
load_dict = checkpoints.restore_checkpoint(ckpt_dir=f"/mnt/data/alzaig/tmp/jax_checkpoints_2", target=None)
rng = load_dict["rng"]
torch.set_rng_state(torch.tensor(load_dict["pytorch_rng"]))
python_state = list(load_dict["python_state"].values())
python_state[1] = tuple(python_state[1].values())
g_train.set_state(torch.tensor(load_dict["generator_dataloader_train"]))
g_test.set_state(torch.tensor(load_dict["generator_dataloader_test"]))
np.random.set_state(tuple(load_dict["numpy_rng"].values()))
random.setstate(python_state)
epoch_last = load_dict["epoch"]
state = flax_serialization.from_state_dict(state, load_dict['state'])

In [ ]:
test_loss, test_accuracy, end_time = eval_model(state.params, test_loader)
print(' test epoch: %d, time: %.4f, loss: %.2f, accuracy: %.2f' % (
      epoch_last, end_time, test_loss, test_accuracy * 100))

for epoch in range(epoch_last+1, n_epochs + 1):
  state, rng = train_epoch(state, train_loader, epoch, rng)
  # Evaluate on the test set after each training epoch
  test_loss, test_accuracy, end_time = eval_model(state.params, test_loader)
  print(' test epoch: %d, time: %.4f, loss: %.2f, accuracy: %.2f' % (
      epoch, end_time, test_loss, test_accuracy * 100))
  # Save the model parameters
  numpy_rng = np.random.get_state()
  python_state = random.getstate()
  torch_state = torch.get_rng_state() # well only this is required but for the sake of completeness I am adding all
  checkpoints.save_checkpoint(
      ckpt_dir="/mnt/data/alzaig/tmp/jax_checkpoints_{}".format(epoch),
      target={"state": state,
              "epoch": epoch,
              "rng": rng,
              "pytorch_rng": torch_state.numpy(),
              "numpy_rng": numpy_rng,
              "python_state": python_state,
              "generator_dataloader_train": g_train.get_state().numpy(),
              "generator_dataloader_test": g_test.get_state().numpy(),
            },
            step=epoch,
            overwrite=True,
    )

Summary time:
- Can we reproduce the results by restarting the script for CPU? Answer ✅.
- Can we reproduce the results by restarting the script for the GPU? Answer ✅
- Can we reproduce the results from the checkpoints? Answer ✅
- Can we reproduce results on CPU and GPU (same device)? Answer ❌
- Can we reproduce results on different CPUs? Answer ❌
- Can we reproduce results on different but same GPU? Answer ✅
- Can we reproduce the results on different graphics cards? Answer ❌ (but I will check this on a few different cards)

### (Optional) Time to clean up before the summary

In [ ]:
# You may want to check if there is nothing valuable in the directory before deleting it
# I even recommend you to do that checking everything in the directory before deleting it
!rm -rf /tmp/*

## Summarize

In this blog, we explored the issue of reproducibility in the popular machine learning frameworks Jax, Tensorflow, and Pytorch. We discussed the challenges and solutions for achieving reproducibility in these frameworks, as well as the benefits of using reproducible methods in machine learning research. We also noted that this blog post did not cover reproducibility in distributed learning environments, which will be the focus of a separate blog post. 

One of the key takeaways from this blog is the importance of setting seeds in order to achieve reproducible results. This is a minimal requirement in the industry, but there are other approaches that can also be useful, particularly in the context of research comparisons. However, it's important to keep in mind that some of these approaches may come at the cost of reduced performance, so they may not be ideal in all situations. That being said, reproducibility can be particularly useful for debugging purposes, even if it may not be the most efficient option in terms of performance.

In this blog, we attempted to summarize the current state of reproducibility in these frameworks, including the ability to reproduce results by restarting the script on both CPU and GPU, using checkpoints, and on different hardware configurations. Please note that this blog is still a work in progress, as we are continuing to research and test these issues on different environments and hardware. If you notice any mistakes or have any additional insights to share, please don't hesitate to contact me, my lord.

## Acknowledgment

I would like to express my gratitude to all the writers and developers who have worked on implementing and documenting reproducible solutions for the various frameworks. In particular, I would like to thank the developers at [OpenAI]((https://openai.com)) for creating ChatGPT, which has made writing this blog much easier.

![chatGPT](../images/posts/chatGPT_meme.png)

## References:

- Python documentation on using the command line (PYTHONHASHSEED): https://docs.python.org/3.3/using/cmdline.html
### Pytorch
- Pytorch documentation on randomness: https://pytorch.org/docs/stable/notes/randomness.html
- Pytorch determinism documentation: https://github.com/NVIDIA/framework-determinism/blob/master/doc/pytorch.md
### Tensorflow
- Tensorflow issue on reproducibility: https://github.com/tensorflow/tensorflow/issues/53771
- Tensorflow documentation on op determinism: https://www.tensorflow.org/api_docs/python/tf/config/experimental/enable_op_determinism
- Tensorflow determinism tutorial: https://suneeta-mall.github.io/2019/12/22/Reproducible-ml-tensorflow.html
- Tensorflow determinism package: https://pypi.org/project/tensorflow-determinism/
- Tensorflow determinism documentation: https://github.com/NVIDIA/framework-determinism/blob/master/doc/tensorflow.md
- Tensorflow determinism status documentation: https://github.com/NVIDIA/framework-determinism/blob/master/doc/tensorflow_status.md
- Tensorflow documentation on threading: https://www.tensorflow.org/api_docs/python/tf/config/threading
### Jax
- Jax documentation on random numbers: https://jax.readthedocs.io/en/latest/jax-101/05-random-numbers.html
- Flax issue on reproducibility: https://github.com/google/flax/issues/33